In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-3.1.0               |           py36_0         724 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         822 KB

The following NEW packages will be INSTALLED:

    altair:  3.1.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forg

In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [8]:
neighborhoods_data = newyork_data['features']

In [9]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [10]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [11]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [12]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [13]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


In [16]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [14]:
CLIENT_ID = 'JPSXBJQ4FURZGDO0OW30YDHOL4UKWQVV1SQWP5UY1HTJNEMN' # your Foursquare ID
CLIENT_SECRET = '2MXC1Z4HHU03WGVPXWQYL05MOELOMIKWHWFQVKUSGSNTC1YG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JPSXBJQ4FURZGDO0OW30YDHOL4UKWQVV1SQWP5UY1HTJNEMN
CLIENT_SECRET:2MXC1Z4HHU03WGVPXWQYL05MOELOMIKWHWFQVKUSGSNTC1YG


In [17]:
manhattan_data.loc[1, 'Neighborhood']

'Chinatown'

In [18]:
neighborhood_latitude = manhattan_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Chinatown are 40.71561842231432, -73.99427936255978.


In [19]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=JPSXBJQ4FURZGDO0OW30YDHOL4UKWQVV1SQWP5UY1HTJNEMN&client_secret=2MXC1Z4HHU03WGVPXWQYL05MOELOMIKWHWFQVKUSGSNTC1YG&v=20180605&ll=40.71561842231432,-73.99427936255978&radius=1000&limit=100'

In [20]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d19e7dc0d2be7002c7888db'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Chinatown',
  'headerFullLocation': 'Chinatown, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 231,
  'suggestedBounds': {'ne': {'lat': 40.72461843131433,
    'lng': -73.98242748932805},
   'sw': {'lat': 40.70661841331431, 'lng': -74.00613123579151}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '578692f4498e1054905dbde7',
       'name': 'Hotel 50 Bowery NYC',
       'location': {'address': '50 Bowery',
        'crossStreet': 'btwn Bayard & Canal St',
        'lat': 40.7159364,
        'lng': -73.9967

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [31]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.address', 'venue.location.crossStreet']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,address,crossStreet
0,Hotel 50 Bowery NYC,Hotel,50 Bowery,btwn Bayard & Canal St
1,Kiki's,Greek Restaurant,130 Division St,btwn Orchard & Ludlow St
2,Bar Belly,Cocktail Bar,14 Orchard St,at Canal St
3,Scarr's Pizza,Pizza Place,22 Orchard St,btwn Canal & Hester St
4,Cheeky Sandwiches,Sandwich Place,35 Orchard St,btwn Canal & Hester St
5,Spicy Village,Chinese Restaurant,68B Forsyth St,btwn Grand & Hester
6,Mission Escape Games,General Entertainment,55 Chrystie St Apt 210,NaN
7,Metrograph,Indie Movie Theater,7 Ludlow St,at Canal St
8,Sam's Fried Ice Cream,Ice Cream Shop,37 Orchard St,Hester St
9,Forgtmenot,New American Restaurant,138 Division St,NaN


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.
